In [1]:
import pandas as pd
import numpy as np
import time
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
import xgboost as xgb
from sklearn import metrics

import os

# 1. Generate Temporal Features

### Step1:  get index_list for get the temporal feature faster

In [2]:
def getNextHourIndexByCurHour(df_day, hour):
    if hour == 20:
        hour = 19
    index_start = (hour - 3 + 1)*548*421
    index_end = (hour - 3 + 2)*548*421
    return range(index_start, index_end)

In [3]:
def getPreviousHourIndexByCurHour(df_day, hour):
    if hour == 3:
        hour = 4
    index_start = (hour - 3 - 1)*548*421
    index_end = (hour - 3)*548*421
    return range(index_start, index_end)

In [4]:
df_rain_day1 = pd.read_csv('../dataset/rainfall_data_day1.csv')

previous_hour_list = []
for hour in range(3, 21):
    previous_hour_list.append(getPreviousHourIndexByCurHour(df_rain_day1, hour))
previous_hour_list = np.ravel(previous_hour_list)

next_hour_list = []
for hour in range(3, 21):
    next_hour_list.append(getNextHourIndexByCurHour(df_rain_day1, hour))
next_hour_list = np.ravel(next_hour_list)

### Step2: Generate Training & Testing Sets Add Temporal Features

In [5]:
for day in range(1, 11):
    if os.path.exists('../dataset/wind_data_add_temporal_feature_day'+ str(day) +'.csv'):
        print ('../dataset/wind_data_add_temporal_feature_day'+ str(day) +'.csv already exists')
        continue
    print(("day {} is begin").format(str(day)))
    start = time.time()
    df_day = pd.read_csv('../dataset/wind_data_day'+ str(day) + '.csv')
    feature = ['predict_' + str(i) for i in range (1, 11, 1)]
    df_day_previous_hour = df_day.iloc[previous_hour_list][feature]
    feature_previous_hour = ['predict_' + str(i) + '_previous_hour' for i in range (1, 11, 1)]
    df_day_previous_hour.columns = feature_previous_hour

    df_day_next_hour = df_day.iloc[next_hour_list][feature]
    feature_next_hour = ['predict_' + str(i) + '_next_hour' for i in range (1, 11, 1)]
    df_day_next_hour.columns = feature_next_hour
                 
    df_day_previous_hour = df_day_previous_hour.reset_index(drop=True)
    df_day_next_hour = df_day_next_hour.reset_index(drop=True)

    df_day_concat = pd.concat([df_day, df_day_previous_hour, df_day_next_hour], axis=1)  
    df_day_concat.to_csv('../dataset/wind_data_add_temporal_feature_day'+ str(day) +'.csv', index=False)
    del df_day, df_day_concat, df_day_previous_hour, df_day_next_hour
    print(("day {} is done").format(str(day)))
    cost_time = time.time() - start
    print(("cost time: {0:.2f} min").format(cost_time/60.0))

../dataset/wind_data_add_temporal_feature_day1.csv already exists
../dataset/wind_data_add_temporal_feature_day2.csv already exists
../dataset/wind_data_add_temporal_feature_day3.csv already exists
../dataset/wind_data_add_temporal_feature_day4.csv already exists
../dataset/wind_data_add_temporal_feature_day5.csv already exists
../dataset/wind_data_add_temporal_feature_day6.csv already exists
../dataset/wind_data_add_temporal_feature_day7.csv already exists
../dataset/wind_data_add_temporal_feature_day8.csv already exists
../dataset/wind_data_add_temporal_feature_day9.csv already exists
../dataset/wind_data_add_temporal_feature_day10.csv already exists


# 2. Train CV Model

In [7]:
for cur_day in range(1, 6):
    df_train = pd.DataFrame()
    for day in [x for x in range(1, 6) if x != cur_day]:
        df_train_day = pd.read_csv('../dataset/wind_data_add_temporal_feature_day'+ str(day) +'.csv')
        print ("day %d is read" % day)
        df_train_day = df_train_day.sample(frac=0.25)
        df_train = pd.concat([df_train, df_train_day], axis= 0, ignore_index=True)
        print ("day %d is concated" % day)
    del df_train_day
    df_test = pd.read_csv('../dataset/wind_data_add_temporal_feature_day' + str(cur_day) + '.csv')
    df_test = df_test.sample(frac=0.25)
    df_train = df_train.drop(['xid', 'yid', 'hour'], axis=1)
    X_train = df_train.drop('real', axis=1).values
    y_train = df_train['real'].values

    df_test = df_test.drop(['xid', 'yid', 'hour'], axis=1)
    X_test = df_test.drop('real', axis=1).values
    y_test = df_test['real'].values
    del df_train, df_test
    xgbReg = XGBRegressor(max_depth=5,
                            learning_rate=0.2,
                            n_estimators=1000,
                            silent=False,
                            objective='reg:linear',
                            nthread=-1,
                            gamma=0,
                            min_child_weight=1,
                            max_delta_step=0,
                            subsample=0.8,
                            colsample_bytree=0.7,
                            colsample_bylevel=1,
                            reg_alpha=0,
                            reg_lambda=1,
                            scale_pos_weight=1,
                            seed=1440,
                            missing=None)


    start = time.time()
    xgbReg.fit(X_train, y_train, eval_metric='rmse', verbose=True, eval_set=[(X_test, y_test)], early_stopping_rounds=20)
    cost_time = time.time() - start
    print("cost time: %s min"  % str(cost_time/60.0))

    with open('../dataset/xgb_time_' + str(cur_day) +'.pickle', 'wb') as f:
        pickle.dump(xgbReg, f)

day 2 is read
day 2 is concated
day 3 is read
day 3 is concated
day 4 is read
day 4 is concated
day 5 is read
day 5 is concated
[0]	validation_0-rmse:10.7779
Will train until validation_0-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:8.63273
[2]	validation_0-rmse:6.91272
[3]	validation_0-rmse:5.55743
[4]	validation_0-rmse:4.47872
[5]	validation_0-rmse:3.62283
[6]	validation_0-rmse:2.95911
[7]	validation_0-rmse:2.43055
[8]	validation_0-rmse:2.02968
[9]	validation_0-rmse:1.72073
[10]	validation_0-rmse:1.49979
[11]	validation_0-rmse:1.34225
[12]	validation_0-rmse:1.23051
[13]	validation_0-rmse:1.15829
[14]	validation_0-rmse:1.11062
[15]	validation_0-rmse:1.08058
[16]	validation_0-rmse:1.06253
[17]	validation_0-rmse:1.05199
[18]	validation_0-rmse:1.04631
[19]	validation_0-rmse:1.04343
[20]	validation_0-rmse:1.04214
[21]	validation_0-rmse:1.04215
[22]	validation_0-rmse:1.04212
[23]	validation_0-rmse:1.04248
[24]	validation_0-rmse:1.04243
[25]	validation_0-rmse:1.04292
[26]	valida

[10]	validation_0-rmse:1.53147
[11]	validation_0-rmse:1.32983
[12]	validation_0-rmse:1.18186
[13]	validation_0-rmse:1.07094
[14]	validation_0-rmse:0.991191
[15]	validation_0-rmse:0.931014
[16]	validation_0-rmse:0.889919
[17]	validation_0-rmse:0.861501
[18]	validation_0-rmse:0.840272
[19]	validation_0-rmse:0.8262
[20]	validation_0-rmse:0.816687
[21]	validation_0-rmse:0.808613
[22]	validation_0-rmse:0.804448
[23]	validation_0-rmse:0.800753
[24]	validation_0-rmse:0.798427
[25]	validation_0-rmse:0.796511
[26]	validation_0-rmse:0.794357
[27]	validation_0-rmse:0.793738
[28]	validation_0-rmse:0.793008
[29]	validation_0-rmse:0.792189
[30]	validation_0-rmse:0.79133
[31]	validation_0-rmse:0.791091
[32]	validation_0-rmse:0.790897
[33]	validation_0-rmse:0.790457
[34]	validation_0-rmse:0.790213
[35]	validation_0-rmse:0.790065
[36]	validation_0-rmse:0.790033
[37]	validation_0-rmse:0.789743
[38]	validation_0-rmse:0.789597
[39]	validation_0-rmse:0.789679
[40]	validation_0-rmse:0.789924
[41]	validation

# 3. predict wind in testing sets

In [8]:
model_paths = ['../dataset/xgb_time_' + str(model) + '.pickle' for model in range(1, 6)]

In [12]:
for day in range(6, 11):
    print ("day %d is begin" % day)
    start = time.time()
    df_test = pd.read_csv('../dataset/wind_data_add_temporal_feature_day'+ str(day) +'.csv')
    df_xgb_space_model = pd.DataFrame()
    df_xgb_space_model = pd.concat([df_xgb_space_model, df_test[['xid', 'yid', 'hour']]], axis=1)
    for model_path in model_paths:
        with open(model_path, 'rb') as f:
            xgb_space_model = pickle.load(f)
        X_train = df_test.drop(['xid', 'yid', 'hour'], axis=1).values
        y_pred = xgb_space_model.predict(X_train)
        column = 'model' + model_path[9:10]
        df_xgb_space_model[column] = y_pred
    df_xgb_space_model.to_csv('../dataset/wind_xgb_cv_time_day'+ str(day) +'.csv', index=False)
    cost = time.time() - start
    print ("cost: %s min" % str(cost/60.0))
    del df_test, df_xgb_space_model, X_train, y_pred

day 6 is begin
cost: 1.402747102578481 min
day 7 is begin
cost: 1.4846476753552755 min
day 8 is begin
cost: 1.3717480778694153 min
day 9 is begin
cost: 1.276911973953247 min
day 10 is begin
cost: 1.3189071496327718 min


# 4. Get the max value of each cv model

In [13]:
for day in range(6, 11):
    df_day = pd.read_csv('../dataset/wind_xgb_cv_time_day'+ str(day) +'.csv')
    cols = df_day.columns
    df_day['predict_final'] = df_day[cols[3:]].max(axis=1)
    df_day.to_csv('../dataset/wind_xgb_cv_time_day'+ str(day) +'.csv', index=False)

# 5. generate wind_matrix

In [14]:
for day in range(6, 11, 1):
    print ('start day{}'.format(day))
    df_test = pd.read_csv('../dataset/wind_xgb_cv_time_day' + str(day) + '.csv')
    for i in range(3, 21):
        t1 = time.time()
        day_hour = df_test[df_test['hour'] == i]
        df_real_day = day_hour.copy()
        xid = df_real_day[df_real_day['hour'] == i]['xid']
        yid = df_real_day[df_real_day['hour'] == i]['yid'] 
        wind = df_real_day[df_real_day['hour'] == i]['predict_final']
        df_test_hour = pd.DataFrame({'xid': list(xid),
                      'yid': list(yid),
                      'wind': list(wind)})
        pt = df_test_hour.pivot_table(index='xid', columns='yid', values='wind', aggfunc=np.sum)
        store_dir = '../dataset/'
        if not os.path.exists(store_dir):
            os.mkdir(store_dir)
        with open(os.path.join(store_dir, 'day' + str(day) + 'hour'+ str(i) +'.pickle'), 'wb') as f:
            pickle.dump(pt, f)
        t2 = time.time()
        print ('cost {0:.3f}s'.format(t2 - t1))

start day6
cost 0.528s
cost 0.388s
cost 0.377s
cost 0.380s
cost 0.382s
cost 0.536s
cost 0.474s
cost 0.421s
cost 0.404s
cost 0.403s
cost 0.429s
cost 0.471s
cost 0.385s
cost 0.433s
cost 0.500s
cost 0.395s
cost 0.383s
cost 0.399s
start day7
cost 0.406s
cost 0.395s
cost 0.396s
cost 0.394s
cost 0.411s
cost 0.412s
cost 0.678s
cost 0.411s
cost 0.414s
cost 0.388s
cost 0.406s
cost 0.393s
cost 0.412s
cost 0.408s
cost 0.404s
cost 0.406s
cost 0.406s
cost 0.445s
start day8
cost 0.401s
cost 0.400s
cost 0.449s
cost 0.396s
cost 0.496s
cost 0.531s
cost 0.686s
cost 0.464s
cost 0.397s
cost 0.436s
cost 0.620s
cost 0.605s
cost 0.456s
cost 0.408s
cost 0.420s
cost 0.393s
cost 0.393s
cost 0.546s
start day9
cost 0.389s
cost 0.382s
cost 0.381s
cost 0.419s
cost 0.397s
cost 0.572s
cost 0.385s
cost 0.400s
cost 0.410s
cost 0.402s
cost 0.427s
cost 0.389s
cost 0.454s
cost 0.401s
cost 0.407s
cost 0.417s
cost 0.405s
cost 0.490s
start day10
cost 0.390s
cost 0.378s
cost 1.069s
cost 0.462s
cost 0.524s
cost 0.407s
cost 0.3

In [15]:
for day in range(6, 11):
    dirpath ='../dataset/'
    wind_matrix = -1
    for hour in range(3, 21, 1):
        filename = 'day'+ str(day) +'hour' + str(hour) + '.pickle'
        with open(os.path.join(dirpath, filename), 'rb') as f:
            matrix = np.array(pickle.load(f, encoding='latin1'))

        if isinstance(wind_matrix, int):
            wind_matrix = matrix[:, :, np.newaxis]
        else:
            wind_matrix = np.concatenate([wind_matrix, matrix[:, :, np.newaxis]], axis=2)
    dirpath ='../dataset/day' + str(day)
    with open(os.path.join(dirpath, 'wind_matrix_xgb.pickle'), 'wb') as f:
        pickle.dump(wind_matrix, f)